# Panel analysis with Python

In [ ]:
import pandas as pd
from linearmodels import PooledOLS # Pooled model
from linearmodels import PanelOLS # Fixed-effect model
from linearmodels import RandomEffects # Random-effect model
from linearmodels.panel import compare # сравнение моделей
import seaborn as sns # data visualization
# conda install -c conda-forge linearmodels

In [ ]:
# Загрузим данные
wages = pd.read_csv('Wages.csv')
wages.head()

In [ ]:
# Размер датасета
wages.shape

In [ ]:
# Преобразуем в формат панельных данных: DataFrame с двумя индексами
# Задаём индексы: сначала индивидуальный, затем временной
wages_panel = wages.set_index(['id', 'time'])
wages_panel

## Wage equation #1 (с количественными)
Рассморим регрессию **lwage на ed, exp, exp^2, wks**

In [ ]:
sns.scatterplot(data=wages, y='lwage', x='ed')

In [ ]:
sns.regplot(data=wages, y='lwage', x='ed', ci=None, line_kws={"color": "r"})

In [ ]:
sns.regplot(data=wages, y='lwage', x='exp', ci=None, line_kws={"color": "r"}, order=2)

### Pooling model

In [ ]:
# Pooling model
pool_mod = PooledOLS.from_formula(formula='lwage~1+ed+exp+I(exp**2)+wks', data=wages_panel).fit()
# Коэффициенты
pool_mod.params

In [ ]:
# Отчёт подгонки
pool_mod.summary

### RE-model

In [ ]:
# RE-model
re_mod = RandomEffects.from_formula(formula='lwage~1+ed+exp+I(exp**2)+wks', data=wages_panel).fit()
# Коэффициенты с округление
re_mod.params.round(3)

In [ ]:
re_mod.summary

### FE-model

In [ ]:
# FE-model
# 'lwage~1+ed+exp+I(exp**2)+wks+EntityEffects' не работает, т.к. ed постоянно во времени!
fe_mod = PanelOLS.from_formula(formula='lwage~1+exp+I(exp**2)+wks+EntityEffects', data=wages_panel).fit()
# Коэффициенты с округление
fe_mod.params.round(3)

In [ ]:
fe_mod.summary

In [ ]:
# Сравнение моделей
compare({'Pool': pool_mod, 'RE': re_mod, 'FE': fe_mod}, start=True)

## Wage equation #2 (с качественными)
Рассморим регрессию **lwage на ed, exp, exp^2, wks, south, smsa**

In [ ]:
# Качественные: 
# Переведем качественные в бинарные
wages = wages.join(pd.get_dummies(wages['south'], prefix='south'), how='right')
wages = wages.join(pd.get_dummies(wages['smsa'], prefix='smsa'), how='right')
wages.drop(['south', 'smsa'], axis=1)
# преобразуем в формат панельных данных
wages_panel = wages.set_index(['id', 'time'])
wages_panel.head()

In [ ]:
# RE-модель
re_mod = RandomEffects.from_formula(formula='lwage~1+ed+exp+I(exp**2)+wks+south_yes+smsa_yes', 
                                    data=wages_panel).fit()
# Коэффициенты с округление
re_mod.params.round(3)

In [ ]:
## FE-модель
fe_mod = PanelOLS.from_formula(formula='lwage~1+exp+I(exp**2)+wks+south_yes+smsa_yes+EntityEffects', 
                                    data=wages_panel).fit()
# Коэффициенты с округление
fe_mod.params.round(3)